<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/%D0%A2%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%82%D0%BE%D1%80%D1%8B%20(%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BPE

In [ ]:
import collections

class BPE:
    """
    Реализация алгоритма Byte-Pair Encoding (BPE) для токенизации текста.
    """
    def __init__(self, vocab_size=None, num_merges=None):
        """
        Инициализирует BPE токенизатор.
        :param vocab_size: Желаемый максимальный размер словаря субтокенов.
                           Если указан, num_merges будет игнорироваться.
        :param num_merges: Количество итераций объединения пар.
                           Если vocab_size не указан, будет использоваться это значение.
        """
        if vocab_size is None and num_merges is None:
            raise ValueError("Необходимо указать либо vocab_size, либо num_merges.")

        self.vocab_size = vocab_size
        self.num_merges = num_merges
        self.merges = {}  # Словарь для хранения выполненных объединений {pair: new_token}
        self.vocabulary = set() # Текущий словарь субтокенов
        self.token_to_id = {} # Словарь для преобразования токенов в ID
        self.id_to_token = [] # Список для преобразования ID в токены

    def _get_initial_tokens(self, text):
        """
        Разбивает текст на начальные символьные токены и добавляет символ конца слова.
        :param text: Входная строка.
        :return: Список списков символьных токенов для каждого слова.
        """
        # Заменяем подчеркивания на пробелы для разделения слов, затем разбиваем по пробелам
        words = text.replace('_', ' ').split()
        initial_tokens = []
        for word in words:
            # Добавляем символ конца слова к каждому слову
            initial_tokens.append(list(word) + ['</w>'])
        return initial_tokens

    def _get_pair_counts(self, tokenized_corpus):
        """
        Подсчитывает частоту встречаемости всех смежных пар токенов в корпусе.
        :param tokenized_corpus: Список списков токенов (представляющих слова).
        :return: Словарь с частотами пар {('token1', 'token2'): count}.
        """
        pair_counts = collections.defaultdict(int)
        for word_tokens in tokenized_corpus:
            # Итерируем по парам токенов в каждом слове
            for i in range(len(word_tokens) - 1):
                pair_counts[(word_tokens[i], word_tokens[i+1])] += 1
        return pair_counts

    def _merge_pair(self, tokenized_corpus, pair, new_token):
        """
        Объединяет все вхождения заданной пары в новый токен в корпусе.
        :param tokenized_corpus: Текущий корпус токенов.
        :param pair: Пара токенов для объединения (tuple).
        :param new_token: Новый токен, который заменит пару.
        :return: Обновленный корпус токенов.
        """
        updated_corpus = []
        for word_tokens in tokenized_corpus:
            new_word_tokens = []
            i = 0
            while i < len(word_tokens):
                # Проверяем, является ли текущая позиция началом искомой пары
                if i + 1 < len(word_tokens) and (word_tokens[i], word_tokens[i+1]) == pair:
                    new_word_tokens.append(new_token)
                    i += 2 # Пропускаем оба токена, которые были объединены
                else:
                    new_word_tokens.append(word_tokens[i])
                    i += 1
            updated_corpus.append(new_word_tokens)
        return updated_corpus

    def fit(self, corpus_text):
        """
        Обучает BPE модель на заданном текстовом корпусе.
        :param corpus_text: Строка, представляющая обучающий корпус.
        """
        # Шаг 1: Инициализация
        # Начальное разбиение на символы и добавление символа конца слова
        current_corpus_tokens = self._get_initial_tokens(corpus_text)

        # Формирование начального словаря из всех уникальных символов
        for word_tokens in current_corpus_tokens:
            for token in word_tokens:
                self.vocabulary.add(token)

        # Определяем количество итераций объединения
        if self.vocab_size is not None:
            # Если задан размер словаря, вычисляем необходимое количество объединений
            # (целевой размер словаря - текущий размер словаря)
            num_iterations = self.vocab_size - len(self.vocabulary)
        elif self.num_merges is not None:
            num_iterations = self.num_merges
        else:
            # Это условие не должно быть достигнуто благодаря проверке в __init__
            num_iterations = 0

        # Шаг 2: Итеративное объединение
        for i in range(num_iterations):
            pair_counts = self._get_pair_counts(current_corpus_tokens)

            if not pair_counts: # Если нет пар для объединения, останавливаемся
                break

            # Находим наиболее частую пару
            # Сортируем по частоте (убывание), затем по лексикографическому порядку (возрастание)
            best_pair = max(pair_counts, key=lambda p: (pair_counts[p], -len(p[0]) - len(p[1]), p))

            # Если частота лучшей пары меньше 1, или нет пар для объединения, останавливаемся
            if pair_counts[best_pair] < 1:
                break

            # Формируем новый токен из объединенной пары
            new_token = "".join(best_pair)

            # Сохраняем объединение
            self.merges[best_pair] = new_token

            # Обновляем корпус, заменяя все вхождения пары новым токеном
            current_corpus_tokens = self._merge_pair(current_corpus_tokens, best_pair, new_token)

            # Добавляем новый токен в словарь
            self.vocabulary.add(new_token)

            # Вывод результатов каждого шага
            print(f"Итерация {i+1}: Объединение '{best_pair[0]}{best_pair[1]}' в '{new_token}'. Частота: {pair_counts[best_pair]}")
            print(f"Обновленный корпус: {current_corpus_tokens}")
            print(f"Текущий словарь: {sorted(list(self.vocabulary))}\n")

        # После обучения создаем отображения токенов в ID и обратно
        self.id_to_token = sorted(list(self.vocabulary))
        self.token_to_id = {token: i for i, token in enumerate(self.id_to_token)}

    def encode(self, text):
        """
        Токенизирует новую строку, используя обученные объединения.
        :param text: Строка для токенизации.
        :return: Список ID токенов.
        """
        # Начальное разбиение текста на символы с добавлением </w>
        words_tokens = self._get_initial_tokens(text)
        encoded_tokens_ids = []

        for word_tokens in words_tokens:
            # Для каждого слова применяем объединения в порядке их обучения
            # Создаем копию списка токенов слова для модификации
            current_word_tokens = list(word_tokens)

            # Применяем объединения итеративно, пока не сможем найти более длинный токен
            # или пока не закончатся объединения
            while True:
                merged_once = False
                new_word_tokens = []
                i = 0
                while i < len(current_word_tokens):
                    found_merge = False
                    # Ищем самое длинное возможное объединение, которое начинается с текущего токена
                    # Проходим по всем известным объединениям в порядке их создания (от коротких к длинным)
                    # или просто ищем самое длинное совпадение
                    best_match_len = 0
                    best_match_token = None

                    # Для простоты, будем применять объединения из self.merges
                    # в порядке их создания (от коротких к длинным)
                    # Это не совсем оптимально, но для демонстрации подходит
                    # В реальных реализациях используют более сложные структуры данных
                    # для эффективного поиска наиболее длинного совпадения
                    for (p1, p2), merged_token in self.merges.items():
                        if i + 1 < len(current_word_tokens) and \
                           current_word_tokens[i] == p1 and \
                           current_word_tokens[i+1] == p2:
                            # Проверяем, является ли это объединение частью более длинного
                            # уже существующего в словаре токена
                            if len(merged_token) > best_match_len:
                                best_match_len = len(merged_token)
                                best_match_token = merged_token
                                found_merge = True

                    if found_merge:
                        new_word_tokens.append(best_match_token)
                        i += 2
                        merged_once = True
                    else:
                        new_word_tokens.append(current_word_tokens[i])
                        i += 1

                if not merged_once:
                    break # Больше нет объединений для этого слова

                current_word_tokens = new_word_tokens

            # Преобразуем токены слова в их ID
            for token in current_word_tokens:
                if token in self.token_to_id:
                    encoded_tokens_ids.append(self.token_to_id[token])
                else:
                    # Если токен не найден (например, из-за OOV или неполного обучения),
                    # разбиваем его на символы и добавляем их ID
                    for char in token:
                        if char in self.token_to_id:
                            encoded_tokens_ids.append(self.token_to_id[char])
                        else:
                            # Этого не должно произойти, если начальный словарь содержит все символы
                            print(f"Предупреждение: Символ '{char}' не найден в словаре.")
                            # Можно добавить токен для неизвестных символов, например, <unk>
        return encoded_tokens_ids

    def decode(self, token_ids):
        """
        Декодирует последовательность ID токенов обратно в строку.
        :param token_ids: Список ID токенов.
        :return: Декодированная строка.
        """
        decoded_tokens = []
        for token_id in token_ids:
            if token_id < len(self.id_to_token):
                decoded_tokens.append(self.id_to_token[token_id])
            else:
                print(f"Предупреждение: ID токена {token_id} вне диапазона словаря.")
                decoded_tokens.append('') # Или можно использовать <unk>

        # Объединяем токены, убираем символ конца слова и заменяем пробелы
        decoded_text = "".join(decoded_tokens).replace('</w>', ' ').strip()
        return decoded_text.replace(' ', '_') # Возвращаем исходный формат с подчеркиваниями

# --- Пример использования ---
if __name__ == "__main__":
    corpus = "Ученик_учится_в_школе,_а_учитель_учит_ученика"

    # Создаем экземпляр BPE, указывая количество объединений
    # Можно также указать vocab_size=X для желаемого размера словаря
    bpe_model = BPE(num_merges=10) # Выполним 10 итераций объединения

    print("--- Обучение BPE модели ---")
    bpe_model.fit(corpus)
    print("\n--- Обучение завершено ---")
    print(f"Финальный словарь BPE (отсортированный): {sorted(list(bpe_model.vocabulary))}")
    print(f"Выполненные объединения: {bpe_model.merges}")

    # Тестирование токенизации
    print("\n--- Тестирование токенизации ---")
    text_to_encode = "Ученик_учится_в_школе,_а_учитель_учит_ученика"
    encoded_ids = bpe_model.encode(text_to_encode)
    print(f"Исходный текст: '{text_to_encode}'")
    print(f"Закодированные ID: {encoded_ids}")

    # Декодирование
    decoded_text = bpe_model.decode(encoded_ids)
    print(f"Декодированный текст: '{decoded_text}'")

    # Проверка на новое слово (OOV)
    print("\n--- Тестирование OOV слова ---")
    oov_text = "Учительница_учит"
    encoded_oov_ids = bpe_model.encode(oov_text)
    print(f"OOV текст: '{oov_text}'")
    print(f"Закодированные ID OOV: {encoded_oov_ids}")
    decoded_oov_text = bpe_model.decode(encoded_oov_ids)
    print(f"Декодированный OOV текст: '{decoded_oov_text}'")

    # Пример токенов по ID
    print("\n--- Токены по ID ---")
    for i, token in enumerate(bpe_model.id_to_token):
        print(f"ID {i}: '{token}'")

#WordPiece

In [ ]:
import collections

class WordPieceTokenizer:
    def __init__(self):
        self.vocab = set()
        self.merges = []

    def preprocess_text(self, text):
        """
        Предварительная обработка текста: приведение к нижнему регистру и разбиение на слова.
        """
        # Удаляем знаки препинания и приводим к нижнему регистру
        processed_text = text.lower().replace('...', '').replace('.', '')
        # Разбиваем на слова по пробелам
        words = processed_text.split(' ')
        # Добавляем пробелы как отдельные токены между словами
        initial_tokens = []
        for i, word in enumerate(words):
            if word: # Убедимся, что слово не пустое
                initial_tokens.extend(list(word))
            if i < len(words) - 1:
                initial_tokens.append(' ') # Добавляем пробел между словами
        return initial_tokens, words # Возвращаем начальные токены и список слов для удобства

    def calculate_frequencies(self, tokens):
        """
        Подсчет частот отдельных токенов (униграмм) и биграмм в текущем корпусе.
        """
        unigram_freq = collections.defaultdict(int)
        bigram_freq = collections.defaultdict(int)

        for i in range(len(tokens)):
            unigram_freq[tokens[i]] += 1
            if i < len(tokens) - 1:
                bigram_freq[(tokens[i], tokens[i+1])] += 1
        return unigram_freq, bigram_freq

    def calculate_score(self, unigram_freq, bigram_freq):
        """
        Вычисление оценки слияния для всех возможных биграмм.
        Score(A, B) = frequency(AB) / (frequency(A) * frequency(B))
        """
        scores = {}
        for (token_a, token_b), freq_ab in bigram_freq.items():
            freq_a = unigram_freq[token_a]
            freq_b = unigram_freq[token_b]
            if freq_a > 0 and freq_b > 0: # Избегаем деления на ноль
                score = freq_ab / (freq_a * freq_b)
                scores[(token_a, token_b)] = score
        return scores

    def merge_tokens(self, tokens, best_bigram):
        """
        Слияние лучшей биграммы в корпусе.
        """
        merged_token = "".join(best_bigram)
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i + 1 < len(tokens) and (tokens[i], tokens[i+1]) == best_bigram:
                new_tokens.append(merged_token)
                i += 2 # Пропускаем оба слитых токена
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens

    def train(self, text, target_vocab_size=None, num_merges=None):
        """
        Обучение WordPiece токенизатора.
        Итеративно сливает токены до достижения целевого размера словаря
        или заданного количества слияний.
        """
        print(f"Исходный текст: '{text}'")
        current_tokens, _ = self.preprocess_text(text)

        # Начальный словарь состоит из всех уникальных символов
        self.vocab = set(current_tokens)
        print(f"\nШаг 1: Начальный словарь (V0) содержит {len(self.vocab)} токенов.")
        print(f"Начальный корпус: {current_tokens}")

        merges_count = 0
        while True:
            unigram_freq, bigram_freq = self.calculate_frequencies(current_tokens)
            scores = self.calculate_score(unigram_freq, bigram_freq)

            if not scores:
                print("\nНет больше биграмм для слияния. Остановка.")
                break

            # Находим биграмму с наивысшей оценкой
            best_bigram = max(scores, key=scores.get)
            best_score = scores[best_bigram]

            merged_token = "".join(best_bigram)

            # Критерий остановки: если новый токен уже в словаре
            if merged_token in self.vocab:
                # Если лучший токен уже существует, удалим его из scores, чтобы найти следующий лучший
                del scores[best_bigram]
                continue # Продолжаем поиск

            # Выводим информацию о текущем слиянии
            print(f"\nИтерация {merges_count + 1}:")
            print(f"  Лучшая биграмма для слияния: '{best_bigram[0]}' + '{best_bigram[1]}' -> '{merged_token}' (Score: {best_score:.4f})")

            # Выполняем слияние в корпусе
            current_tokens = self.merge_tokens(current_tokens, best_bigram)

            # Добавляем новый токен в словарь и сохраняем слияние
            self.vocab.add(merged_token)
            self.merges.append(best_bigram)
            merges_count += 1

            print(f"  Корпус после слияния: {current_tokens}")
            print(f"  Текущий размер словаря: {len(self.vocab)}")

            # Критерии остановки
            if target_vocab_size is not None and len(self.vocab) >= target_vocab_size:
                print(f"\nДостигнут целевой размер словаря ({target_vocab_size}). Остановка.")
                break
            if num_merges is not None and merges_count >= num_merges:
                print(f"\nДостигнуто заданное количество слияний ({num_merges}). Остановка.")
                break

        print(f"\nОбучение завершено. Итоговый словарь содержит {len(self.vocab)} токенов.")
        return sorted(list(self.vocab))

    def tokenize(self, text):
        """
        Токенизация нового текста с использованием обученного словаря WordPiece.
        """
        if not hasattr(self, 'vocab') or len(self.vocab) == 0:
            raise ValueError("Токенизатор не обучен. Сначала вызовите метод train().")

        print(f"\n--- Токенизация нового текста ---")
        print(f"Текст для токенизации: '{text}'")

        # Предварительная обработка текста для токенизации
        processed_text = text.lower().replace('...', '').replace('.', '')
        words = processed_text.split(' ')

        final_tokens = []
        for word in words:
            if not word: # Пропускаем пустые строки, если они возникли из-за множественных пробелов
                continue

            word_tokens = []
            remaining_word = word

            while remaining_word:
                found_match = False
                # Ищем самый длинный подтокен из словаря, который является префиксом оставшейся части слова
                for i in range(len(remaining_word), 0, -1):
                    subword = remaining_word[:i]

                    if subword in self.vocab:
                        word_tokens.append(subword)
                        remaining_word = remaining_word[i:]
                        found_match = True
                        break

                if not found_match:
                    # Если не удалось найти соответствующий токен, разбиваем на символы
                    # или используем токен [UNK]
                    if remaining_word[0] in self.vocab: # Если символ есть в словаре
                        word_tokens.append(remaining_word[0])
                    else: # Если символ даже не в начальном словаре
                        word_tokens.append('[UNK]')
                    remaining_word = remaining_word[1:]

            final_tokens.extend(word_tokens)
            final_tokens.append(' ') # Добавляем пробел между токенизированными словами

        # Удаляем последний пробел, если он есть
        if final_tokens and final_tokens[-1] == ' ':
            final_tokens.pop()

        print(f"Токенизированный текст: {final_tokens}")
        return final_tokens

    def get_vocab(self):
        """Возвращает текущий словарь токенов."""
        return sorted(list(self.vocab))

    def get_merges(self):
        """Возвращает список выполненных слияний."""
        return self.merges

# --- Пример использования ---
if __name__ == "__main__":
    text_to_train = "В училище учитель учит ученикам по новому учебнику"

    # Создаем экземпляр токенизатора
    tokenizer = WordPieceTokenizer()

    # Обучаем токенизатор
    final_vocab = tokenizer.train(text_to_train, num_merges=50)

    print(f"\nИтоговый словарь WordPiece: {final_vocab}")

    # Токенизируем тот же текст
    tokenizer.tokenize(text_to_train)

    # Пример токенизации нового слова
    new_text = "учительница"
    # Добавим '##ница' в словарь для демонстрации
    if 'ница' not in final_vocab:
        tokenizer.vocab.add('ница')
        tokenizer.vocab.add('##ница')
        final_vocab = tokenizer.get_vocab()

    print(f"\nИтоговый словарь WordPiece (с 'ница' для демонстрации): {final_vocab}")
    tokenizer.tokenize(new_text)

    # Демонстрация с упрощенным словарем
    simplified_tokenizer = WordPieceTokenizer()
    simplified_tokenizer.vocab = set(['у', 'ч', 'и', 'т', 'е', 'л', 'ь', 'н', 'ц', 'а', 'учитель', '##ница', '##тель', '##а'])
    print(f"\n--- Демонстрация токенизации подслов с упрощенным словарем ---")
    simplified_tokenizer.tokenize("учительница")

# Униграмной языковой модели

In [ ]:
import re
from collections import Counter

def tokenize(text):
    """
    Токенизирует текст, приводя его к нижнему регистру и удаляя знаки препинания.
    Args:
        text (str): Входной текст.
    Returns:
        list: Список токенов (слов).
    """
    # Приводим текст к нижнему регистру
    text = text.lower()
    # Удаляем знаки препинания и разбиваем на слова
    # Используем re.findall для извлечения только буквенных последовательностей
    tokens = re.findall(r'\b[а-яёa-z]+\b', text)
    return tokens

def train_unigram_model(corpus_text):
    """
    Обучает униграмную языковую модель на заданном корпусе текста.
    Рассчитывает вероятности слов методом максимального правдоподобия (MLE)
    и со сглаживанием по Лапласу.
    Args:
        corpus_text (str): Текст корпуса для обучения модели.
    Returns:
        tuple: Кортеж, содержащий:
            - dict: Вероятности слов по MLE.
            - dict: Вероятности слов со сглаживанием по Лапласу.
            - int: Общее количество слов в корпусе (N).
            - int: Размер словаря (количество уникальных слов).
    """
    # Шаг 1: Токенизация корпуса
    tokens = tokenize(corpus_text)

    # Общее количество слов в корпусе (N)
    N = len(tokens)
    if N == 0:
        print("Ошибка: Корпус пуст после токенизации.")
        return {}, {}, 0, 0

    # Шаг 2: Подсчет частот слов
    word_counts = Counter(tokens)

    # Шаг 3: Определение словаря и его размера
    vocabulary = list(word_counts.keys())
    V = len(vocabulary)

    # Шаг 4: Расчет вероятностей слов (MLE)
    mle_probabilities = {}
    for word, count in word_counts.items():
        mle_probabilities[word] = count / N

    # Расчет вероятностей слов со сглаживанием по Лапласу
    laplace_probabilities = {}
    for word in vocabulary:
        laplace_probabilities[word] = (word_counts[word] + 1) / (N + V)
    # Для слов, которых нет в словаре, но могут встретиться в новом тексте
    # Их вероятность со сглаживанием будет (0 + 1) / (N + V)
    # Мы не добавляем их сюда напрямую, но это учитывается при использовании модели.

    print(f"Общее количество слов в корпусе (N): {N}")
    print(f"Размер словаря (|V|): {V}")
    print("\nЧастоты слов:")
    for word, count in word_counts.items():
        print(f"  '{word}': {count}")
    print("\nВероятности слов (MLE):")
    for word, prob in mle_probabilities.items():
        print(f"  P('{word}') = {prob:.4f}")
    print("\nВероятности слов (Laplace Smoothing):")
    for word, prob in laplace_probabilities.items():
        print(f"  P_Laplace('{word}') = {prob:.4f}")

    return mle_probabilities, laplace_probabilities, N, V

def calculate_sentence_probability(sentence, model_probabilities, N, V, smoothing_type='laplace'):
    """
    Рассчитывает вероятность предложения, используя обученную униграмную модель.
    Args:
        sentence (str): Предложение для оценки.
        model_probabilities (dict): Словарь вероятностей слов (MLE или Laplace).
        N (int): Общее количество слов в обучающем корпусе.
        V (int): Размер словаря обучающего корпуса.
        smoothing_type (str): Тип сглаживания ('none' для MLE, 'laplace' для сглаживания по Лапласу).
    Returns:
        float: Вероятность предложения.
    """
    sentence_tokens = tokenize(sentence)
    probability = 1.0

    print(f"\nОценка вероятности предложения: '{sentence}'")
    print(f"Токены предложения: {sentence_tokens}")

    for token in sentence_tokens:
        if smoothing_type == 'laplace':
            # Используем формулу Лапласа для каждого токена
            # Если токена нет в model_probabilities (т.е. Count(token) = 0),
            # то его вероятность будет (0 + 1) / (N + V)
            word_count_in_model = model_probabilities.get(token, 0) * N # Восстанавливаем Count(token)
            # Если слово не было в словаре, то word_count_in_model будет 0.0,
            # но для Лапласа нам нужен его реальный счетчик (0).
            # Проще использовать прямой расчет:
            if token in model_probabilities:
                p_token = (word_count_in_model + 1) / (N + V)
            else:
                p_token = 1 / (N + V) # Для неизвестных слов
            print(f"  P_Laplace('{token}') = {p_token:.8f}")
        else: # Без сглаживания (MLE)
            p_token = model_probabilities.get(token, 0.0) # Если слово не найдено, вероятность 0
            print(f"  P_MLE('{token}') = {p_token:.8f}")

        if p_token == 0:
            # Если вероятность слова 0, то вероятность всего предложения 0
            probability = 0.0
            print(f"  Обнаружено неизвестное слово '{token}' без сглаживания. Вероятность предложения = 0.")
            break
        probability *= p_token

    return probability

# --- Пример использования ---
corpus_text = "Ученики пишут диктант. Учитель диктует. Мы пишем, он пишет, они пишут. Пишите внимательно."

print("--- Обучение Униграмной модели ---")
mle_probs, laplace_probs, N_corpus, V_corpus = train_unigram_model(corpus_text)

# Оценка вероятности предложения из корпуса (без сглаживания)
sentence1 = "Ученики пишут диктант"
prob_sentence1_mle = calculate_sentence_probability(sentence1, mle_probs, N_corpus, V_corpus, smoothing_type='none')
print(f"\nВероятность предложения '{sentence1}' (MLE): {prob_sentence1_mle:.15f}")

# Оценка вероятности предложения из корпуса (со сглаживанием по Лапласу)
prob_sentence1_laplace = calculate_sentence_probability(sentence1, laplace_probs, N_corpus, V_corpus, smoothing_type='laplace')
print(f"\nВероятность предложения '{sentence1}' (Laplace Smoothing): {prob_sentence1_laplace:.15f}")

# Оценка вероятности нового предложения с неизвестным словом (без сглаживания)
sentence2 = "Учитель читает книгу"
prob_sentence2_mle = calculate_sentence_probability(sentence2, mle_probs, N_corpus, V_corpus, smoothing_type='none')
print(f"\nВероятность предложения '{sentence2}' (MLE): {prob_sentence2_mle:.15f}")

# Оценка вероятности нового предложения с неизвестным словом (со сглаживанием по Лапласу)
prob_sentence2_laplace = calculate_sentence_probability(sentence2, laplace_probs, N_corpus, V_corpus, smoothing_type='laplace')
print(f"\nВероятность предложения '{sentence2}' (Laplace Smoothing): {prob_sentence2_laplace:.15f}")

# Пример с одним словом, которого нет в корпусе
unknown_word_sentence = "школа"
prob_unknown_laplace = calculate_sentence_probability(unknown_word_sentence, laplace_probs, N_corpus, V_corpus, smoothing_type='laplace')
print(f"\nВероятность слова '{unknown_word_sentence}' (Laplace Smoothing): {prob_unknown_laplace:.15f}")


# SentencePiece

In [ ]:
# Импортируем библиотеку SentencePiece
import sentencepiece as spm
import os # Для работы с файловой системой

print("--- Полный пример использования SentencePiece ---")

# Шаг 1: Установка SentencePiece (инструкция)
# Если у вас еще не установлен SentencePiece, вы можете установить его с помощью pip:
# pip install sentencepiece
# (Раскомментируйте строку выше и запустите в терминале, если необходимо)

# Шаг 2: Подготовка данных для обучения
# Для обучения SentencePiece требуется текстовый файл.
# Создадим пример файла с текстом, который мы будем использовать.
# Используем тот же текст, что и в лекции.
text_data = """Учитель говорит: «Откройте учебники». Ученики открывают учебники. Откройте страницу 15. Страница сложная. Страницы учебника содержат много информации."""

# Определяем имя файла для обучения
train_file_name = "train_text_for_sp.txt"

# Создаем файл для обучения и записываем в него текст
try:
    with open(train_file_name, "w", encoding="utf-8") as f:
        f.write(text_data)
    print(f"\nСоздан файл для обучения: '{train_file_name}'")
except IOError as e:
    print(f"Ошибка при создании файла для обучения: {e}")
    exit() # Выходим, если не удалось создать файл

# Шаг 3: Обучение модели SentencePiece
# Мы будем обучать модель с использованием алгоритма Unigram (по умолчанию).
# model_prefix: префикс для файлов модели (model.model и model.vocab)
# vocab_size: желаемый размер словаря подслов
# character_coverage: процент символов, которые должны быть покрыты моделью
#                  (1.0 означает, что все символы из входных данных должны быть представлены)
# model_type: 'unigram' (по умолчанию) или 'bpe'
# input: путь к файлу(ам) для обучения

model_prefix = "my_custom_sentencepiece_model"
vocab_size = 50 # Увеличим размер словаря для лучшей демонстрации

print(f"\nНачинается обучение модели SentencePiece с vocab_size={vocab_size}...")
try:
    spm.SentencePieceTrainer.train(
        input=train_file_name,
        model_prefix=model_prefix,
        vocab_size=vocab_size,
        character_coverage=1.0,
        model_type='unigram' # Можно изменить на 'bpe'
    )
    print(f"Обучение завершено. Созданы файлы: '{model_prefix}.model' и '{model_prefix}.vocab'")
except Exception as e:
    print(f"Ошибка при обучении модели SentencePiece: {e}")
    # Попытаемся удалить созданный файл, если обучение не удалось
    if os.path.exists(train_file_name):
        os.remove(train_file_name)
    exit()

# Шаг 4: Загрузка обученной модели
# После обучения мы можем загрузить модель для использования.
sp = spm.SentencePieceProcessor()
model_path = f"{model_prefix}.model"

try:
    sp.load(model_path)
    print(f"\nМодель SentencePiece загружена: '{model_path}'")
except Exception as e:
    print(f"Ошибка при загрузке модели SentencePiece: {e}")
    # Попытаемся удалить созданные файлы, если загрузка не удалась
    if os.path.exists(train_file_name): os.remove(train_file_name)
    if os.path.exists(f"{model_prefix}.model"): os.remove(f"{model_prefix}.model")
    if os.path.exists(f"{model_prefix}.vocab"): os.remove(f"{model_prefix}.vocab")
    exit()

# Шаг 5: Токенизация текста (кодирование)
# Теперь используем загруженную модель для токенизации нашего текста.

input_text = "Учитель говорит: «Откройте учебники». Ученики открывают учебники. Откройте страницу 15. Страница сложная. Страницы учебника содержат много информации."

print(f"\n--- Токенизация ---")
print(f"Исходный текст: '{input_text}'")

# Кодирование в ID токенов
encoded_ids = sp.encode_as_ids(input_text)
print(f"Токенизация в ID: {encoded_ids}")

# Кодирование в строковые токены (подслова)
encoded_pieces = sp.encode_as_pieces(input_text)
print(f"Токенизация в подслова: {encoded_pieces}")

# Шаг 6: Детокенизация текста (декодирование)
# Восстановим исходный текст из токенизированных ID или подслов.

print(f"\n--- Детокенизация ---")

decoded_text_from_ids = sp.decode_ids(encoded_ids)
print(f"Детокенизация из ID: '{decoded_text_from_ids}'")

decoded_text_from_pieces = sp.decode_pieces(encoded_pieces)
print(f"Детокенизация из подслов: '{decoded_text_from_pieces}'")

# Проверка обратимости
if input_text == decoded_text_from_ids and input_text == decoded_text_from_pieces:
    print("\nПроцесс токенизации и детокенизации полностью обратим!")
else:
    print("\nВнимание: Детокенизированный текст не полностью совпадает с исходным.")
    print(f"Ожидалось: '{input_text}'")
    print(f"Получено из ID: '{decoded_text_from_ids}'")
    print(f"Получено из подслов: '{decoded_text_from_pieces}'")


# Шаг 7: Очистка (удаление созданных файлов)
# Удалим файлы, созданные в процессе обучения, чтобы не засорять рабочую директорию.
print(f"\n--- Очистка временных файлов ---")
try:
    if os.path.exists(train_file_name):
        os.remove(train_file_name)
        print(f"Удален файл: '{train_file_name}'")
    if os.path.exists(f"{model_prefix}.model"):
        os.remove(f"{model_prefix}.model")
        print(f"Удален файл: '{model_prefix}.model'")
    if os.path.exists(f"{model_prefix}.vocab"):
        os.remove(f"{model_prefix}.vocab")
        print(f"Удален файл: '{model_prefix}.vocab'")
except OSError as e:
    print(f"Ошибка при удалении файлов: {e}")

print("\n--- Использование SentencePiece завершено ---")
